In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col



# function for batch 
def process_and_save_batch_data():
    try:
        # reading data 
        customers_df = spark.read.table("raw_data.customers")
        branches_df = spark.read.table("raw_data.branches")
        
        # saving to bronze
        customers_df.write.format("delta").mode("overwrite").saveAsTable("Bronze_layer.customers")
        branches_df.write.format("delta").mode("overwrite").saveAsTable("Bronze_layer.branches")

        print("Batch data processed and loaded to the bronze.")
    except Exception as e:
        print(f"Error in batch processing: {e}")

# function for streaming
def ingest_transactions():
    try:
        # read streaming
        transactions_stream_df = spark.readStream \
            .table("raw_data.transactions")

        # writing stream
        query = transactions_stream_df.writeStream.format("delta").option("checkpointLocation", "Bronze_layer.checkpoint").outputMode("append").toTable("Bronze_layer.transactions")

       
        print("streaming started...")
        return query
    except Exception as e:
        print(f"Error starting streaming: {e}")
        return None

# running batch function 
process_and_save_batch_data()

# running streaming function
query = ingest_transactions()
if query:
    try:
        query.awaitTermination()
    except KeyboardInterrupt:
        print("streaming interrupted.")
else:
    print("streaming  not working.")


Batch data processed and loaded to the bronze.
streaming started...
